# Generating trading signals with Gradient Boosting

This notebook illustrates the following steps:
1. **Cross-validation in the time-series context** poses the additional challenge that train and validation sets need to respect the temporal order of the data so that we do not inadvertently train the model on data 'from the future' to predict the past and introduce lookahead bias. Scikit-learn's built-in [TimeSeriesSplit](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.TimeSeriesSplit.html) aims to accomplish this but does not work for this case where we have multiple time series, one for each ticker. We could solve this by manually subsetting the data for the appropriate train and validation periods. Alternatively, we can create a custom time-series splitter compatible with the scikit-learn Kfold interface (see resources). Here is an [example](https://github.com/stefan-jansen/machine-learning-for-trading/blob/master/utils.py) that illustrates how to do so for this case. This allows us to specify fixed `train_length` and `test_length` parameters, as well as a `lookahead` value that defines the forecast horizon and ensures an appropriate gap between the training and validation set.
2. Now, we'll investigate if a more complex **gradient boosting** model is capable of improving the result. 
    - We'll use the LightGBM implementation because it often [outperforms](https://lightgbm.readthedocs.io/en/latest/Experiments.html) the alternative [XGBoost](https://xgboost.readthedocs.io/en/latest/) and [CatBoost](https://catboost.ai/) libraries, especially in terms of runtime and memory footprint. There is also a recent scikit-learn [HistGradientBoostingRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.HistGradientBoostingRegressor.html#sklearn.ensemble.HistGradientBoostingRegressor) inspired by LightGBM. 
    - You need to define and [tune](https://lightgbm.readthedocs.io/en/latest/Parameters-Tuning.html) several **hyperparameters** to define the model's complexity as well as constraints on the learning process (see [docs](https://lightgbm.readthedocs.io/en/latest/Parameters.html) for the full list). The goal is to adjust the model's capacity to the data while limiting the risk of overfitting so that the model learns the signal rather than the noise in the data and generalizes better to out-of-sample data. In other words, we aim to identify the most promising model settings based on an unbiased estimate of the generalization error, and then apply these settings to generate out-of-sample predictions.
    - The **most important parameters** define the *number of iterations* or trees, and the *size of each tree*. LightGBM allows you to optimize the number of iterations by training a model for a certain number of iterations, then evaluating its performance before continuing to train the model. LightGBM permits [leaf-wise tree growth](https://lightgbm.readthedocs.io/en/latest/Features.html#leaf-wise-best-first-tree-growth) and let's you limit the size of the tree by setting the number of leaf nodes or by limiting the depth of the tree. A useful combination targets a certain number of leaves while constraining the minimum number of samples per leaf node to avoid excessive tree imbalances as well as noisy estimates due to leaf nodes with few samples. The default value of 32 of the number of leaves is a good starting point, but the min. number of data points per leaves should be significantly higher than the default of 20 - with 1m observations and 100 leave nodes, each leaf would on average contain 10K samples. 
    - A lower learning rate can help boost accuracy but requires more iterations and, thus, extends training time. See also LightGBM's [guidance](https://lightgbm.readthedocs.io/en/latest/Parameters-Tuning.html) on parameter tuning.
    - Finally, the parameter `feature_frac` let's you control the amount of randomization while building each tree.
3. To **evaluate the model performance**, compare the cross-validation IC for different hyperparameter settings, computed again as the average of the Spearman rank correlation of the model predictions for each day with the actual returns. Identify the best-performing 5-10 models over a number of recent quarters, and check whether averaging their predictions further improves the result. In addition, use Alphalens to compute a summary tearsheet and inspect the spread between the botton and the top quintile of the predictions.
4. Generate out-of-sample predictions for 2014-2016 using your preferred models (which may change for every quarter), and possibly average the result if you found this to yield more stable results. Repeat your Alphalens analysis for these signals.

> If you have a GPU, you can install the LightGBM GPU version instead of the CPU version to get a [decent speedup](https://lightgbm.readthedocs.io/en/latest/GPU-Performance.html). To accomplish this, run
```bash
conda remove -n liveproject lightgbm -y
pip install lightgbm --install-option=--gpu
```
You will also need to adapt the LightGBM parameter settings below to indicate that you'll be using the `device` `GPU`. A smaller bin size like 63 tends to perform better than the default 255.

## Imports

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
%matplotlib inline

from pathlib import Path
import sys, os
from time import time
from collections import defaultdict
from itertools import product

import numpy as np
import pandas as pd
import statsmodels.api as sm

import lightgbm as lgb

from scipy.stats import spearmanr

from alphalens.tears import (create_summary_tear_sheet,
                             create_full_tear_sheet)

from alphalens.utils import get_clean_factor_and_forward_returns

import matplotlib.pyplot as plt
import seaborn as sns

## Settings

In [3]:
sns.set_style('whitegrid')

In [4]:
np.random.seed(42)

In [5]:
idx = pd.IndexSlice

## Get Data

In [8]:
DATA_PATH = Path('..', 'data')

## Custom Time Series Cross-Validation

## Model Selection: Lookback, lookahead and roll-forward periods

## LightGBM Model Tuning

## Generate Predictions for best models

## AlphaLens Analysis